In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [2]:
training_data_uri = 's3://pwalanki-blunder-predictor-data/output-2020-march'


In [3]:
!pygmentize 'model.py'

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import numpy as np
import argparse
import time
import os
import json
import math


def model(training_data, validation_data, dense_layers, num_nodes, epochs, learning_rate, dropout, callback=None):
	feature_columns = [
		tf.feature_column.numeric_column('position', shape=(1, 8, 8, 12), dtype=tf.dtypes.int64),
		tf.feature_column.numeric_column('turn', shape=(1,), dtype=tf.dtypes.int64),
		tf.feature_column.numeric_column('elo', shape=(1,), dtype=tf.dtypes.float32)
	]

	model = tf.keras.models.Sequential()
	model.add(tf.keras.layers.DenseFeatures(feature_columns))

	for i in range(dense_layers):
		model.add(tf.keras.layers.Dense(num_nodes, activation=tf.nn.relu))

	model.add(tf.keras.layers.Dropout(dropout))

	model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

	model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
				  loss='binary_crossentropy

In [4]:
from sagemaker.tensorflow import TensorFlow


metric_definitions = [
    {
        'Name': 'val_loss',
        'Regex': 'val_loss: ([0-9\\.]+)'
    },
    {
        'Name': 'val_accuracy',
        'Regex': 'val_accuracy: ([0-9\\.]+)'
    },
    {
        'Name': 'val_recall',
        'Regex': 'val_recall: ([0-9\\.]+)'
    },
    {
        'Name': 'val_precision',
        'Regex': 'val_precision: ([0-9\\.]+)'
    }
]

hyperparameters = {
    'aws': True,
    'batch-size': 64,
    'epochs': 1000,
    'dense-layers': 4,
    'dropout': 0.2,
    'learning-rate': 0.0000001,
    'num-nodes': 2048,
    'parts': 208,
    'early-stopping': True  
}

estimator = TensorFlow(entry_point='model.py',
                       role=role,
                       train_instance_count=1,
                       train_instance_type='ml.m5.xlarge',
                       framework_version='2.1.0',
                       py_version='py3',
                       metric_definitions=metric_definitions,
                       hyperparameters=hyperparameters)

In [ ]:
estimator.fit(training_data_uri)

2020-04-09 04:02:08 Starting - Starting the training job...
2020-04-09 04:02:11 Starting - Launching requested ML instances.